In [18]:
import os
import numpy as np
import rasterio
from rasterio.features import geometry_mask
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
import geopandas as gpd

In [19]:
ROOT = r"C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO"
VARS = ["CBD", "CBH", "CC", "CH"]

LAKE_GPKG  = r"C:\Users\bsf31\Documents\data\NL060\LFAPI\Vector\firesheds_fire_scars.gpkg"
LAKE_LAYER = "lake2024_5070"

OUT_DIR = os.path.join(ROOT, "_max_value")
os.makedirs(OUT_DIR, exist_ok=True)

In [20]:
FNAME_BY_YEAR = {
    "2024": "Landfire_LF250US_250{VAR}_SBCO.tif",
    "2023": "Landfire_LF240US_240{VAR}_SBCO.tif",  # used ONLY inside Lake polygon
    "2016": "Landfire_LF200US_200{VAR}_SBCO.tif",
    "2001": "us_105{VAR}_SBCO.tif",
}

In [21]:
def read_band(path):
    ds = rasterio.open(path)
    arr = ds.read(1, masked=True)  # masked by nodata
    return ds, arr

def read_aligned_to_template(path, template_ds):
    """Read single band aligned to template grid without writing to disk."""
    with rasterio.open(path) as src:
        with WarpedVRT(
            src,
            crs=template_ds.crs,
            transform=template_ds.transform,
            width=template_ds.width,
            height=template_ds.height,
            resampling=Resampling.nearest,
            nodata=template_ds.nodata,
        ) as vrt:
            arr = vrt.read(1, masked=True)
    return arr

def write_like(template_ds, out_path, data, dtype=None):
    prof = template_ds.profile.copy()
    prof.update(count=1, tiled=True, compress="None", BIGTIFF="IF_SAFER")
    if dtype is not None:
        prof.update(dtype=dtype)
    with rasterio.open(out_path, "w", **prof) as dst:
        if isinstance(data, np.ma.MaskedArray) and template_ds.nodata is not None:
            dst.write(data.filled(template_ds.nodata), 1)
        else:
            dst.write(data, 1)

def main():
    lake = gpd.read_file(LAKE_GPKG, layer=LAKE_LAYER)

    for VAR in VARS:
        var_dir = os.path.join(ROOT, VAR)
        paths = {yr: os.path.join(var_dir, patt.format(VAR=VAR))
                 for yr, patt in FNAME_BY_YEAR.items()}

        # Template: 2024 (defines grid)
        ds_2024, a2024_raw = read_band(paths["2024"])
        nodata = ds_2024.nodata

        # Align everyone to 2024’s exact grid (handles subtle 1-pixel diffs)
        a2024 = a2024_raw  # already on template
        a2016 = read_aligned_to_template(paths["2016"], ds_2024)
        a2001 = read_aligned_to_template(paths["2001"], ds_2024)
        a2023 = read_aligned_to_template(paths["2023"], ds_2024)

        # Step 1: MAX across 2001, 2016, 2024
        max_2016_2024 = np.ma.maximum(a2016, a2024)
        max_all = np.ma.maximum(a2001, max_2016_2024)

        # Lake mask (True = inside)
        lake_in_raster_crs = lake.to_crs(ds_2024.crs)
        geoms = list(lake_in_raster_crs.geometry)
        lake_mask = ~geometry_mask(
            geoms,
            transform=ds_2024.transform,
            invert=False,
            out_shape=(ds_2024.height, ds_2024.width),
            all_touched=False
        )

        # Step 2: Overwrite inside Lake with 2023 wherever 2023 has valid data
        overwrite_cond = lake_mask & (~a2023.mask)
        out = np.ma.array(max_all, copy=True)
        out.data[overwrite_cond] = a2023.data[overwrite_cond]
        out.mask[overwrite_cond] = False  # ensure visible

        # Write
        out_path = os.path.join(OUT_DIR, f"SBCO_{VAR}_max_2001_2016_2023Lake_2024.tif")
        write_like(ds_2024, out_path, out, dtype=out.dtype)
        print(f"[OK] {VAR} -> {out_path}")

        ds_2024.close()

In [22]:
if __name__ == "__main__":
    main()

[OK] CBD -> C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_value\SBCO_CBD_max_2001_2016_2023Lake_2024.tif
[OK] CBH -> C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_value\SBCO_CBH_max_2001_2016_2023Lake_2024.tif
[OK] CC -> C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_value\SBCO_CC_max_2001_2016_2023Lake_2024.tif
[OK] CH -> C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_value\SBCO_CH_max_2001_2016_2023Lake_2024.tif


In [28]:
# -*- coding: utf-8 -*-
# From-scratch: Landfire ImageServer (LF250, LF240, LF200) + local US_105
# Max(LF250, LF200, US_105), then overwrite inside Lake with LF240, then final clip.
# Outputs pyramids using provided write_raster_with_pyramids().
#
# pip install geopandas shapely rasterio numpy requests

import os
import io
import math
import json
import requests
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from rasterio.features import geometry_mask
from rasterio.io import MemoryFile
from shapely.ops import unary_union
from shapely.geometry import box

# --------------------------
# CONFIG
# --------------------------
# Variables to process
VARS = ["CBD", "CBH", "CC", "CH"]

# Landfire ImageServer endpoints (variable placeholder {VAR})
# (CBD example provided by you; pattern repeated for other vars)
LF250_URL = "https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF250/US_250{VAR}/ImageServer"
LF240_URL = "https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF240/US_240{VAR}/ImageServer"
LF200_URL = "https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200{VAR}_20/ImageServer"

# Local 2001 paths (adjust names if your filenames differ per variable)
LOCAL_2001_DIR = r"C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\RAW\2001"
LOCAL_2001_FILES = {
    "CBD": os.path.join(LOCAL_2001_DIR, r"US_105_CBD_20250916\US_105_CBD_20250916\US_105_CBD\Tif\us_105cbd.tif"),
    "CBH": os.path.join(LOCAL_2001_DIR, r"US_105_CBH_20250916\US_105_CBH_20250916\US_105_CBH\Tif\us_105cbh.tif"),
    "CC" : os.path.join(LOCAL_2001_DIR, r"US_105_CC_20250916\US_105_CC_20250916\US_105_CC\Tif\us_105cc.tif"),
    "CH" : os.path.join(LOCAL_2001_DIR, r"US_105_CH_20250916\US_105_CH_20250916\US_105_CH\Tif\us_105ch.tif"),
}

# Clip geometries (source GPKG layers in EPSG:4269)
GPKG = r"C:\Users\bsf31\Documents\data\NL060\LFAPI\Vector\firesheds_fire_scars.gpkg"
LAKE_LAYER = "lake2024_4269"
FINAL_CLIP_LAYER = "sbco_firesheds_v04a_2025922_epsg4269"

# Target processing CRS (use Landfire native: EPSG:5070 – CONUS Albers)
PROC_EPSG = 4269

# Output directory
OUT_DIR = r"C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_from_services"
os.makedirs(OUT_DIR, exist_ok=True)

# Desired pixel size in target CRS (Landfire is 30 m at 250 versions)
PIXEL_SIZE = 30.0  # meters

# --------------------------
# UTILITIES
# --------------------------
def export_image(image_server_url, bbox_5070, width, height, out_nodata=-9999, pixel_type="F32"):
    """Export a GeoTIFF from ArcGIS ImageServer to memory on a fixed grid."""
    url = f"{image_server_url}/exportImage"
    xmin, ymin, xmax, ymax = bbox_5070
    params = {
        "bbox": f"{xmin},{ymin},{xmax},{ymax}",
        "bboxSR": PROC_EPSG,
        "imageSR": PROC_EPSG,
        "size": f"{width},{height}",
        "format": "tiff",
        "pixelType": pixel_type,
        "noData": out_nodata,
        "noDataInterpretation": "esriNoDataMatchAny",
        "interpolation": "RSP_NearestNeighbor",
        "f": "image",
    }
    r = requests.get(url, params=params, timeout=180)
    r.raise_for_status()
    mem = MemoryFile(io.BytesIO(r.content))
    return mem

def geometry_union_in_crs(gdf, epsg):
    g = gdf.to_crs(epsg).geometry
    return unary_union([geom for geom in g if geom is not None])

def rasterize_mask(template_ds, geom):
    mask = ~geometry_mask(
        [geom],
        out_shape=(template_ds.height, template_ds.width),
        transform=template_ds.transform,
        invert=False,
        all_touched=False,
    )
    return mask  # True inside geometry

def to_masked(arr, nodata):
    return np.ma.masked_where(arr == nodata, arr) if nodata is not None else np.ma.masked_invalid(arr)

def write_tif_like(template_ds, out_path, data, nodata=32767, dtype="Int16", compress="None"):
    """Write a single-band GeoTIFF matching the template grid/CRS."""
    profile = template_ds.profile.copy()
    profile.update(
        driver="GTiff",
        count=1,
        dtype=dtype,
        compress=compress,
        tiled=True,
        BIGTIFF="IF_SAFER",
        nodata=nodata,
    )
    with rasterio.open(out_path, "w", **profile) as dst:
        if isinstance(data, np.ma.MaskedArray):
            dst.write(data.filled(nodata).astype(dtype), 1)
        else:
            dst.write(data.astype(dtype), 1)

# --------------------------
# PIPELINE
# --------------------------
def main():
    # Read clip layers (4269), prep geometries in 5070
    lake_gdf  = gpd.read_file(GPKG, layer=LAKE_LAYER)
    final_gdf = gpd.read_file(GPKG, layer=FINAL_CLIP_LAYER)
    lake_geom_5070  = geometry_union_in_crs(lake_gdf,  PROC_EPSG)
    final_geom_5070 = geometry_union_in_crs(final_gdf, PROC_EPSG)

    # Build template processing grid from final clip bounds
    minx, miny, maxx, maxy = final_geom_5070.bounds
    width  = int(math.ceil((maxx - minx) / PIXEL_SIZE))
    height = int(math.ceil((maxy - miny) / PIXEL_SIZE))
    maxx = minx + width  * PIXEL_SIZE
    maxy = miny + height * PIXEL_SIZE
    bbox_5070 = (minx, miny, maxx, maxy)

    for VAR in VARS:
        print(f"\n--- {VAR} ---")

        # Exports (LF250 & LF200) on the exact grid
        mem250 = export_image(LF250_URL.format(VAR=VAR), bbox_5070, width, height)
        mem200 = export_image(LF200_URL.format(VAR=VAR), bbox_5070, width, height)
        with mem250.open() as ds250, mem200.open() as ds200:
            a250 = ds250.read(1)  # float32
            a200 = ds200.read(1)
            nodata = ds250.nodata if ds250.nodata is not None else 32767

            # Local US_105 aligned on-the-fly to the same grid
            path105 = LOCAL_2001_FILES[VAR]
            if not os.path.exists(path105):
                raise FileNotFoundError(f"Missing 2001 raster for {VAR}: {path105}")
            with rasterio.open(path105) as src105, WarpedVRT(
                src105,
                crs=ds250.crs,
                transform=ds250.transform,
                width=ds250.width,
                height=ds250.height,
                resampling=Resampling.nearest,
                nodata=nodata,
            ) as vrt105:
                a105 = vrt105.read(1)

            # Masked arrays for max op
            A250 = to_masked(a250, nodata)
            A200 = to_masked(a200, nodata)
            A105 = to_masked(a105, nodata)

            base_max = np.ma.maximum(np.ma.maximum(A250, A200), A105)

            # Export LF240 (2023), same grid, then overwrite inside Lake
            mem240 = export_image(LF240_URL.format(VAR=VAR), bbox_5070, width, height)
            with mem240.open() as ds240:
                a240 = ds240.read(1)
            A240 = to_masked(a240, nodata)

            lake_mask = rasterize_mask(ds250, lake_geom_5070)
            overwrite = lake_mask & (~A240.mask)

            out = np.ma.array(base_max, copy=True)
            out.data[overwrite] = A240.data[overwrite]
            out.mask[overwrite] = False  # ensure visible

            # Final clip (mask outside final firesheds)
            final_mask = rasterize_mask(ds250, final_geom_5070)  # True inside
            out.mask = np.where(final_mask, out.mask, True)      # outside -> masked

            # Write plain GeoTIFF (no pyramids)
            out_path = os.path.join(
                OUT_DIR, f"SBCO_{VAR}_max250_200_105_240Lake.tif"
            )
            write_tif_like(ds250, out_path, out, nodata=nodata, dtype="float32", compress="LZW")
            print(f"✓ wrote {out_path}")

        # Ensure MemoryFiles are closed (context managers above handle it)
        mem250.close(); mem200.close(); mem240.close()

    print("\nDone.")

if __name__ == "__main__":
    main()



--- CBD ---
✓ wrote C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_from_services\SBCO_CBD_max250_200_105_240Lake.tif

--- CBH ---
✓ wrote C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_from_services\SBCO_CBH_max250_200_105_240Lake.tif

--- CC ---
✓ wrote C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_from_services\SBCO_CC_max250_200_105_240Lake.tif

--- CH ---
✓ wrote C:\Users\bsf31\Documents\data\NL060\LFAPI\Tiff\SBCO\_max_from_services\SBCO_CH_max250_200_105_240Lake.tif

Done.
